# HapTree-X experimental workbook

## Tools

The following tools were used for experiment preparation:

| Tool | Version |
|------|-----|
| HapTree-X | [b87ea39fa8](https://github.com/LillianZ/HapTreeX/tree/b87ea39fa8024d966ebe5ad027918c65053b6db2) |
| STAR | [2.6.1d](https://github.com/alexdobin/STAR/archive/2.6.1d.tar.gz) |
| HapCUT2 | [782e537](https://github.com/vibansal/HapCUT2/tree/782e537a66b0372983e873573ed7616511825a77) |
| HapCUT | [0.7 (254c473)](https://github.com/vibansal/hapcut/tree/254c473066565e6b9d11b0e725e17ff910dcfbb1) |
| GATK | [3.8-1-0 (gf15c1c3ef)](https://software.broadinstitute.org/gatk/download/auth?package=GATK-archive&version=3.8-1-0-gf15c1c3ef) |
| phASER | [1.1.0 (b7c28f4)](https://github.com/secastel/phaser/tree/b7c28f4e37790e7df4f41cf6bf74f9d5f41f7d0c) |


## File hierarchy

- `deniz_old`: whatever Deniz did previously
- `resources`: Reference genome (GRCh37), genome annotations and STAR reference genome
- `tools`: location of the tools used in these experiments
- `samples/SAMPLE`: sample data
  - `giab`: Genome in the Bottle data
    - `SAMPLE.vcf`: original GIAB VCF
      - `SAMPLE.filter.vcf`: GIAB VCF with only heterozygous diploid SNPs
      - `SAMPLE.blind.vcf`: Filtered GIAB VCF with any extra information (including phasing!) stripped out
    - `SAMPLE.hs37d5.wgs.bam`: Whole genome BAM file from GIAB FTP repository
    - `SAMPLE.grch73.wxs.bam`: Exome BAM file from GIAB FTP repository
  - `rna`: RNA-seq data (backed up on `/data/cb/`)
    - `fastq`: Original RNA-seq FASTQs from Sarah
    - `SAMPLE.star.sam`: STAR-produced SAM
    - `SAMPLE.name-clean.sam`: filtered and name-sorted STAR SAM
    - `SAMPLE.name-clean.bam`: filtered and position-sorted STAR BAM
  - `phaser`: phASER output on `SAMPLE.name-clean.bam`
    - `SAMPLE.<xxx>`: various phASER files
  - `hapcut`: HapCUT-related files
    - `SAMPLE-rna.chair`: HAIRS from RNA-seq BAM 
    - `SAMPLE-wxs.chair`: HAIRS from Exome GIAB BAM 
    - `SAMPLE-wgs.chair`: HAIRS from WGS GIAB BAM 
  - `haptreex`: HapTree-X-related files
    - `SAMPLE-rna.chair`: Chair output from RNA-seq BAM 
    - `rna`: HapTree-X output with `--RNAfragmat SAMPLE-rna.chair`  
- `tmp`: triaging directory

## Data preparation

### GIAB data

To download GIAB data run:

```bash
# Do the following 2 steps for each ${sample} :
cd samples/${sample}
mkdir -p giab

# Get WGS BAMs
wget -c -O giab/NA12878.hs37d5.wgs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/NA12878/NIST_NA12878_HG001_HiSeq_300x/NHGRI_Illumina300X_novoalign_bams/HG001.hs37d5.300x.bam
wget -c -O giab/NA24385.hs37d5.wgs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG002_NA24385_son/NIST_HiSeq_HG002_Homogeneity-10953946/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG002.hs37d5.300x.bam
wget -c -O giab/NA24149.hs37d5.wgs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG003_NA24149_father/NIST_HiSeq_HG003_Homogeneity-12389378/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG003.hs37d5.300x.bam 
wget -c -O giab/NA24143.hs37d5.wgs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG004_NA24143_mother/NIST_HiSeq_HG004_Homogeneity-14572558/NHGRI_Illumina300X_AJtrio_novoalign_bams/HG004.hs37d5.300x.bam
wget -c -O giab/NA24631.hs37d5.wgs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/ChineseTrio/HG005_NA24631_son/HG005_NA24631_son_HiSeq_300x/NHGRI_Illumina300X_Chinesetrio_novoalign_bams/HG005.hs37d5.300x.bam

parallel 'samtools index {}' ::: *.bam

# Get WXS BAMs
wget -c -O giab/NA24631.hg19.wxs.NIST7035-1.bam \
  https://ftp-trace.ncbi.nih.gov/giab/ftp/data/NA12878/Garvan_NA12878_HG001_HiSeq_Exome/project.NIST_NIST7035_H7AP8ADXX_TAAGGCGA_1_NA12878.bwa.markDuplicates.bam
wget -c -O giab/NA24631.hg19.wxs.NIST7035-2.bam \
  https://ftp-trace.ncbi.nih.gov/giab/ftp/data/NA12878/Garvan_NA12878_HG001_HiSeq_Exome/project.NIST_NIST7035_H7AP8ADXX_TAAGGCGA_2_NA12878.bwa.markDuplicates.bam
wget -c -O giab/NA24631.hg19.wxs.NIST7086-1.bam \
  https://ftp-trace.ncbi.nih.gov/giab/ftp/data/NA12878/Garvan_NA12878_HG001_HiSeq_Exome/project.NIST_NIST7086_H7AP8ADXX_CGTACTAG_1_NA12878.bwa.markDuplicates.bam
wget -c -O giab/NA24631.hg19.wxs.NIST7086-2.bam \
  https://ftp-trace.ncbi.nih.gov/giab/ftp/data/NA12878/Garvan_NA12878_HG001_HiSeq_Exome/project.NIST_NIST7086_H7AP8ADXX_CGTACTAG_2_NA12878.bwa.markDuplicates.bam

wget -c -O giab/NA24385.grch37.wxs.bam \
    ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG002_NA24385_son/OsloUniversityHospital_Exome/151002_7001448_0359_AC7F6GANXX_Sample_HG002-EEogPU_v02-KIT-Av5_AGATGTAC_L008.posiSrt.markDup.bam	
wget -c -O giab/NA24149.grch37.wxs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG003_NA24149_father/OsloUniversityHospital_Exome/151002_7001448_0359_AC7F6GANXX_Sample_HG003-EEogPU_v02-KIT-Av5_TCTTCACA_L008.posiSrt.markDup.bam	
wget -c -O giab/NA24143.grch37.wxs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG004_NA24143_mother/OsloUniversityHospital_Exome/151002_7001448_0359_AC7F6GANXX_Sample_HG004-EEogPU_v02-KIT-Av5_CCGAAGTA_L008.posiSrt.markDup.bam	
wget -c -O giab/NA24631.grch37.wxs.bam \
  ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/ChineseTrio/HG005_NA24631_son/OsloUniversityHospital_Exome/151002_7001448_0359_AC7F6GANXX_Sample_HG005-EEogPU_v02-KIT-Av5_CGCATACA_L008.posiSrt.markDup.bam

# Get VCFs
wget -c -O giab/NA12878.vcf.gz \
  https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/NA12878_HG001/latest/GRCh37/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz
wget -c -O giab/NA24385.vcf.gz \
  https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG002_NA24385_son/latest/GRCh37/HG002_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-22_v.3.3.2_highconf_triophased.vcf.gz
wget -c -O giab/NA24149.vcf.gz \
  https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG003_NA24149_father/latest/GRCh37/HG003_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X_CHROM1-22_v.3.3.2_highconf.vcf.gz
wget -c -O giab/NA24143.vcf.gz \
  https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/AshkenazimTrio/HG004_NA24143_mother/latest/GRCh37/HG004_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X_CHROM1-22_v.3.3.2_highconf.vcf.gz
wget -c -O giab/NA24631.vcf.gz \
  https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/ChineseTrio/HG005_NA24631_son/latest/GRCh37/HG005_GRCh37_highconf_CG-IllFB-IllGATKHC-Ion-SOLID_CHROM1-22_v.3.3.2_highconf.vcf.gz
```

#### Preparing GIAB BAMs and VCFs for phasing

To clean-up GIAB data and prepare it for phasing run:

```bash
for sample in NA12878 NA24143 NA24149 NA24385 NA24631 ;
do
  echo -e "\n${sample}"
  echo "====================================================="

  # Filter out anything that is not heterozygous biallelic SNP
  bcftools view samples/${sample}/giab/${sample}.vcf.gz \
    -g 'het' -m2 -M2 -v snps \
    -o samples/${sample}/giab/${sample}.filter.vcf
    
  # Remove phasing and other annotations from VCF to "blind" it
  bcftools annotate -x 'INFO,FMT' samples/${sample}/giab/${sample}.filter.vcf | \
    awk '{if (substr($0,0,1)!="#") { $10="0/1"; } print}' OFS="\t" \
    > samples/${sample}/giab/${sample}.blind.vcf
    
  # Create indices
  bgzip -c samples/${sample}/giab/${sample}.filter.vcf \
    > samples/${sample}/giab/${sample}.filter.vcf.gz
  bgzip -c samples/${sample}/giab/${sample}.blind.vcf \
    > samples/${sample}/giab/${sample}.blind.vcf.gz
  parallel --bar tabix -p vcf {} ::: samples/${sample}/giab/*.vcf.gz
done
```

### STAR RNA-seq alignment

#### Reference preparation

To prepare STAR-compatible Human reference run:

```bash
mkdir -p resources 

wget -c -P resources \
  ftp://ftp.ensembl.org/pub/grch37/release-94/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz
wget -c -P resources \ 
  ftp://ftp.ensembl.org/pub/grch37/release-94/gtf/homo_sapiens/Homo_sapiens.GRCh37.87.gtf.gz

gunzip resources/Homo_sapiens.GRCh37.87.gtf.gz
gunzip resources/Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz
samtools faidx resources/Homo_sapiens.GRCh37.dna.primary_assembly.fa

# Reference generation (takes around 1/2 hr)
tools/star-v2.6.1d/bin/Linux_x86_64/STAR \
  --runMode genomeGenerate \
  --runThreadN 10 \
  --genomeDir resources/Homo_sapiens.GRCh37 \
  --genomeFastaFiles resources/Homo_sapiens.GRCh37.dna.primary_assembly.fa  \
  --sjdbGTFfile resources/Homo_sapiens.GRCh37.87.gtf \
  --sjdbOverhang 74
```

#### Sample alignment

To align each RNA-seq sample run:

```bash
for sample in NA12878 NA24143 NA24149 NA24385 NA24631 ;
do
  r1="$(ls samples/NA12878/rna/fastq/*R1*.gz | tr '\n' ',')"
  r2="$(ls samples/NA12878/rna/fastq/*R2*.gz | tr '\n' ',')"

  # Alignment: should take ~10 min and use ~40G of RAM
  mesa tools/star-v2.6.1d/bin/Linux_x86_64/STAR \
    --runThreadN 24 \
    --twopassMode Basic \
    --genomeDir resources/Homo_sapiens.GRCh37 \
    --readFilesCommand gunzip -c \
    --outFileNamePrefix "samples/${sample}/rna/star/${sample}_" \
    --readFilesIn "${r1}" "${r2}";
    
  mv samples/${sample}/rna/star/${sample}_Aligned.out.sam \
     samples/${sample}/rna/${sample}.star.sam
  # Prepare CHAIR-compatible SAMs
  grep -v "^@" \
    samples/${sample}/rna/${sample}.star.sam | \
    sort -s -k1,1 | \
    awk "$5 == 255" | \
    > samples/${sample}/rna/${sample}.name-clean.sam
  
  # Prepare BAMs for phASER
  samtools view -b -@4 samples/${sample}/rna/${sample}.name-clean.sam | \
    samtools sort -@3 -o samples/${sample}/rna/${sample}.name-clean.bam 
  samtools index samples/${sample}/rna/${sample}.name-clean.bam
    
  # [optional] rm -r samples/${sample}/rna/star 
done
```
 
A successful run's output should look like:
```
Nov 21 21:11:05 ..... started STAR run
Nov 21 21:11:05 ..... loading genome
Nov 21 21:11:30 ..... started 1st pass mapping
Nov 21 21:14:43 ..... finished 1st pass mapping
Nov 21 21:14:44 ..... inserting junctions into the genome indices
Nov 21 21:16:39 ..... started mapping
Nov 21 21:20:53 ..... finished successfully
Time: 589.14 s
Mem:  39716452 Kb
```

## Phasing 

### Phasing with HapTree-X

#### Chair output preparation

```bash
for sample in NA12878 NA24143 NA24149 NA24385 NA24631 ; 
do
  mkdir -p samples/${sample}/haptreex
  mesa tools/chair-deniz/chair \
    samples/${sample}/giab/${sample}.blind.vcf \
    samples/${sample}/rna/${sample}.name-clean.sam \
    samples/${sample}/haptreex/${sample}-rna.chair \
    1 TRANS_FILTER PCR_DUP_FILTER CONFLICT_FILTER 
done
```

A single run takes less than 5 minutes and expected output is something like:
```
OPT flag: TRANS_FILTER
OPT flag: PCR_DUP_FILTER
OPT flag: CONFLICT_FILTER
numLinesProcessed: 20000000 [...]
Time: 160.73 s
Mem:  99952 Kb
```

#### Run HapTree-X in DASE mode

```bash
for sample in NA12878 NA24143 NA24149 NA24385 NA24631 ; 
do
  echo -e "\n${sample}";
  echo "=====================================================";

  mesa python2 tools/haptreex-master/HapTree.py \
    --vcf=samples/${sample}/giab/${sample}.blind.vcf \
    --RNAfragmat=samples/${sample}/haptreex/${sample}-rna.chair \
    --gene_info=resources/Homo_sapiens.GRCh37.87.gtf \
    --outputfolder=samples/${sample}/haptreex/rna
done
```

Output on empty kraken:
```
NA12878
=====================================================
Loading VCF file for scoring
Loading and formatting fragments
1008715 reads of sufficient quality
47358 distinct reads
Loading VCF file
Preparing data for ReadGraph
Loading and formatting genes
running through genes
graph made
assigning rates
rates assigned
('Pt 1 took ', 46.92121696472168)
('stats took ', 0.018706083297729492)
Making ReadGraph
9876 SNPs in non-trivial connected components
('Pt 2 took ', 0.177872896194458)
('Pt 3 took ', 0.23997712135314941)
('Pt 4 took ', 0.02936100959777832)
('Total took ', 47.387134075164795)
Time: 50.36 s
Mem:  4994404 Kb

NA24143
=====================================================
Loading VCF file for scoring
Loading and formatting fragments
1117697 reads of sufficient quality
38120 distinct reads
Loading VCF file
Preparing data for ReadGraph
Loading and formatting genes
running through genes
graph made
assigning rates
rates assigned
('Pt 1 took ', 41.883825063705444)
('stats took ', 0.01724696159362793)
Making ReadGraph
7909 SNPs in non-trivial connected components
('Pt 2 took ', 0.22679901123046875)
('Pt 3 took ', 0.1896979808807373)
('Pt 4 took ', 0.0278470516204834)
('Total took ', 42.34541606903076)
Time: 45.44 s
Mem:  4930284 Kb

NA24149
=====================================================
Loading VCF file for scoring
Loading and formatting fragments
1305232 reads of sufficient quality
37270 distinct reads
Loading VCF file
Preparing data for ReadGraph
Loading and formatting genes
running through genes
graph made
assigning rates
rates assigned
('Pt 1 took ', 42.62701988220215)
('stats took ', 0.017560958862304688)
Making ReadGraph
6994 SNPs in non-trivial connected components
('Pt 2 took ', 0.1820070743560791)
('Pt 3 took ', 0.16524100303649902)
('Pt 4 took ', 0.021738052368164062)
('Total took ', 43.013566970825195)
Time: 45.67 s
Mem:  4916524 Kb

NA24385
=====================================================
Loading VCF file for scoring
Loading and formatting fragments
945888 reads of sufficient quality
65160 distinct reads
Loading VCF file
Preparing data for ReadGraph
Loading and formatting genes
running through genes
graph made
assigning rates
rates assigned
('Pt 1 took ', 41.47194004058838)
('stats took ', 0.020869970321655273)
Making ReadGraph
13255 SNPs in non-trivial connected components
('Pt 2 took ', 0.273906946182251)
('Pt 3 took ', 0.3198421001434326)
('Pt 4 took ', 0.03827786445617676)
('Total took ', 42.124836921691895)
Time: 44.75 s
Mem:  5002228 Kb

NA24631
=====================================================
Loading VCF file for scoring
Loading and formatting fragments
1610738 reads of sufficient quality
68997 distinct reads
Loading VCF file
Preparing data for ReadGraph
Loading and formatting genes
running through genes
graph made
assigning rates
rates assigned
('Pt 1 took ', 48.58053517341614)
('stats took ', 0.023324012756347656)
Making ReadGraph
14110 SNPs in non-trivial connected components
('Pt 2 took ', 1.5242679119110107)
('Pt 3 took ', 0.3374600410461426)
('Pt 4 took ', 0.04451608657836914)
('Total took ', 50.51010322570801)
Time: 53.26 s
Mem:  4978432 Kb
```

### Running HapCUT

#### Data preparation

Create HAIRS for HapCUT(2):

```bash
for sample in NA12878 NA24143 NA24149 NA24385 NA24631 ; 
do
  echo -e "\n${sample}"
  echo "====================================================="
  mkdir -p samples/${sample}/hapcut
  
  # RNA: takes ~1m/sample
  mesa tools/hapcut2-master/build/extractHAIRS \
    --bam samples/${sample}/rna/${sample}.name-clean.bam  \
    --vcf samples/${sample}/giab/${sample}.blind.vcf \
    --mbq 0 --mmq 0 --maxIS 1000000000 --minIS 0 \
    --out samples/${sample}/hapcut/${sample}-rna.chair
    
  # WXS: takes ~3m/sample
  mesa tools/hapcut2-master/build/extractHAIRS \
    --bam samples/${sample}/giab/${sample}.grch37.wxs.bam  \
    --vcf samples/${sample}/giab/${sample}.blind.vcf \
    --out samples/${sample}/hapcut/${sample}-wxs.chair

  # WGS: takes ~hour+/sample
  mesa tools/hapcut2-master/build/extractHAIRS \
    --bam samples/${sample}/giab/${sample}.hs37d5.wgs.bam  \
    --vcf samples/${sample}/giab/${sample}.blind.vcf \
    --out samples/${sample}/hapcut/${sample}-wgs.chair
done
```

### Running phASER

phASER is ran as follows:

```bash
for id sample in 1 NA12878 2 NA24385 3 NA24149 4 NA24143 5 NA24631 ; 
do
  echo -e "\n${sample} (HG00${id})"
  echo "====================================================="
  mkdir -p samples/${sample}/phaser
  
  mesa python2 tools/phaser-v1.1.0/phaser/phaser.py \
    --vcf samples/${sample}/giab/${sample}.blind.vcf.gz \
    --bam samples/${sample}/rna/${sample}.name-clean.bam \
    --paired_end 1 --mapq 255 --baseq 0 \
    --sample HG00${id} \
    --o samples/${sample}/phaser/${sample}
done
```

Output on empty kraken:
```
NA12878 (HG001)
=====================================================

##################################################
              Welcome to phASER v1.1.0
  Author: Stephane Castel (scastel@nygenome.org)
  Updated by: Bishwa K. Giri (bkgiri@uncg.edu)
##################################################

Completed the check of dependencies and input files availability...

STARTED "Read backed phasing and ASE/haplotype analyses" ...
    DATE, TIME : 2019-01-07, 19:45:06
#1. Loading heterozygous variants into intervals...
Processing sample named HG001
    using all the chromosomes ...
    processing VCF...

    Memory efficient mode is deactivated...
    If RAM is limited, activate memory efficient mode using the flag "--process_slow = 1"...

     creating variant mapping table...
          1968169 heterozygous sites being used for phasing (0 filtered, 0 indels excluded, 1968169 unphased)

#2. Retrieving reads that overlap heterozygous sites...
     file: samples/NA12878/rna/NA12878.name-clean.bam
          minimum mapq: 255
          mapping reads to variants...
               completed chromosome 1...
               completed chromosome 2...
               completed chromosome 3...
               completed chromosome 4...
               completed chromosome 5...
               completed chromosome 6...
               completed chromosome 7...
               completed chromosome 8...
               completed chromosome 9...
               completed chromosome 10...
               completed chromosome 11...
               completed chromosome 12...
               completed chromosome 13...
               completed chromosome 14...
               completed chromosome 15...
               completed chromosome 16...
               completed chromosome 17...
               completed chromosome 18...
               completed chromosome 19...
               completed chromosome 20...
               completed chromosome 21...
               completed chromosome 22...
               completed chromosome X...
          processing mapped reads...
          using alignment score cutoff of 122
          retrieved 1201340 reads
#3. Identifying connected variants...
     calculating sequencing noise level...
     sequencing noise level estimated at 0.000533
     creating read sets...
     generating read connectivity map...
     testing variant connections versus noise...
     87 variant connections dropped because of conflicting configurations (threshold = 0.010000)
     35232 variants covered by at least 1 read
#4. Identifying haplotype blocks...
#5. Phasing blocks...
#6. Outputting haplotypes...
#7. Outputting phased VCF...
     GT field is not being updated with phASER genome wide phase. This can be changed using the --gw_phase_vcf argument.
     Compressing and tabix indexing output VCF...

     COMPLETED using 1201340 reads in 593 seconds using 1 threads
     PHASED  9519 of 1968169 all variants (= 0.004836) with at least one other variant
     GENOME WIDE PHASED  0 of 1968169 unphased variants (= 0.000000)
     GENOME WIDE PHASE CORRECTED  0 of 1968169 variants (= 0.000000)
     Global maximum memory usage: 1175.47 (mb)

COMPLETED "Read backed phasing" of sample HG001 in 00:09:54 hh:mm:ss
DATE, TIME : 2019-01-07, 19:55:00

The End.
Time: 597.79 s
Mem:  1203684 Kb

NA24385 (HG002)
=====================================================

##################################################
              Welcome to phASER v1.1.0
  Author: Stephane Castel (scastel@nygenome.org)
  Updated by: Bishwa K. Giri (bkgiri@uncg.edu)
##################################################

Completed the check of dependencies and input files availability...

STARTED "Read backed phasing and ASE/haplotype analyses" ...
    DATE, TIME : 2019-01-07, 19:55:03
#1. Loading heterozygous variants into intervals...
Processing sample named HG002
    using all the chromosomes ...
    processing VCF...

    Memory efficient mode is deactivated...
    If RAM is limited, activate memory efficient mode using the flag "--process_slow = 1"...

     creating variant mapping table...
          1901907 heterozygous sites being used for phasing (0 filtered, 0 indels excluded, 1901907 unphased)

#2. Retrieving reads that overlap heterozygous sites...
     file: samples/NA24385/rna/NA24385.name-clean.bam
          minimum mapq: 255
          mapping reads to variants...
               completed chromosome 1...
               completed chromosome 2...
               completed chromosome 3...
               completed chromosome 4...
               completed chromosome 5...
               completed chromosome 6...
               completed chromosome 7...
               completed chromosome 8...
               completed chromosome 9...
               completed chromosome 10...
               completed chromosome 11...
               completed chromosome 12...
               completed chromosome 13...
               completed chromosome 14...
               completed chromosome 15...
               completed chromosome 16...
               completed chromosome 17...
               completed chromosome 18...
               completed chromosome 19...
               completed chromosome 20...
               completed chromosome 21...
               completed chromosome 22...
          processing mapped reads...
          using alignment score cutoff of 122
          retrieved 1070439 reads
#3. Identifying connected variants...
     calculating sequencing noise level...
     sequencing noise level estimated at 0.000491
     creating read sets...
     generating read connectivity map...
     testing variant connections versus noise...
     115 variant connections dropped because of conflicting configurations (threshold = 0.010000)
     49065 variants covered by at least 1 read
#4. Identifying haplotype blocks...
#5. Phasing blocks...
#6. Outputting haplotypes...
#7. Outputting phased VCF...
     GT field is not being updated with phASER genome wide phase. This can be changed using the --gw_phase_vcf argument.
     Compressing and tabix indexing output VCF...

     COMPLETED using 1070439 reads in 626 seconds using 1 threads
     PHASED  12789 of 1901907 all variants (= 0.006724) with at least one other variant
     GENOME WIDE PHASED  0 of 1901907 unphased variants (= 0.000000)
     GENOME WIDE PHASE CORRECTED  0 of 1901907 variants (= 0.000000)
     Global maximum memory usage: 1137.36 (mb)

COMPLETED "Read backed phasing" of sample HG002 in 00:10:27 hh:mm:ss
DATE, TIME : 2019-01-07, 20:05:30

The End.
Time: 629.82 s
Mem:  1164660 Kb

NA24149 (HG003)
=====================================================

##################################################
              Welcome to phASER v1.1.0
  Author: Stephane Castel (scastel@nygenome.org)
  Updated by: Bishwa K. Giri (bkgiri@uncg.edu)
##################################################

Completed the check of dependencies and input files availability...

STARTED "Read backed phasing and ASE/haplotype analyses" ...
    DATE, TIME : 2019-01-07, 20:05:33
#1. Loading heterozygous variants into intervals...
Processing sample named HG003
    using all the chromosomes ...
    processing VCF...

    Memory efficient mode is deactivated...
    If RAM is limited, activate memory efficient mode using the flag "--process_slow = 1"...

     creating variant mapping table...
          1854441 heterozygous sites being used for phasing (0 filtered, 0 indels excluded, 1854441 unphased)

#2. Retrieving reads that overlap heterozygous sites...
     file: samples/NA24149/rna/NA24149.name-clean.bam
          minimum mapq: 255
          mapping reads to variants...
               completed chromosome 1...
               completed chromosome 2...
               completed chromosome 3...
               completed chromosome 4...
               completed chromosome 5...
               completed chromosome 6...
               completed chromosome 7...
               completed chromosome 8...
               completed chromosome 9...
               completed chromosome 10...
               completed chromosome 11...
               completed chromosome 12...
               completed chromosome 13...
               completed chromosome 14...
               completed chromosome 15...
               completed chromosome 16...
               completed chromosome 17...
               completed chromosome 18...
               completed chromosome 19...
               completed chromosome 20...
               completed chromosome 21...
               completed chromosome 22...
          processing mapped reads...
          using alignment score cutoff of 116
          retrieved 1490547 reads
#3. Identifying connected variants...
     calculating sequencing noise level...
     sequencing noise level estimated at 0.000458
     creating read sets...
     generating read connectivity map...
     testing variant connections versus noise...
     256 variant connections dropped because of conflicting configurations (threshold = 0.010000)
     28111 variants covered by at least 1 read
#4. Identifying haplotype blocks...
#5. Phasing blocks...
#6. Outputting haplotypes...
#7. Outputting phased VCF...
     GT field is not being updated with phASER genome wide phase. This can be changed using the --gw_phase_vcf argument.
     Compressing and tabix indexing output VCF...

     COMPLETED using 1490547 reads in 683 seconds using 1 threads
     PHASED  6442 of 1854441 all variants (= 0.003474) with at least one other variant
     GENOME WIDE PHASED  0 of 1854441 unphased variants (= 0.000000)
     GENOME WIDE PHASE CORRECTED  0 of 1854441 variants (= 0.000000)
     Global maximum memory usage: 1149.44 (mb)

COMPLETED "Read backed phasing" of sample HG003 in 00:11:24 hh:mm:ss
DATE, TIME : 2019-01-07, 20:16:57

The End.
Time: 688.35 s
Mem:  1177024 Kb

NA24143 (HG004)
=====================================================

##################################################
              Welcome to phASER v1.1.0
  Author: Stephane Castel (scastel@nygenome.org)
  Updated by: Bishwa K. Giri (bkgiri@uncg.edu)
##################################################

Completed the check of dependencies and input files availability...

STARTED "Read backed phasing and ASE/haplotype analyses" ...
    DATE, TIME : 2019-01-07, 20:17:01
#1. Loading heterozygous variants into intervals...
Processing sample named HG004
    using all the chromosomes ...
    processing VCF...

    Memory efficient mode is deactivated...
    If RAM is limited, activate memory efficient mode using the flag "--process_slow = 1"...

     creating variant mapping table...
          1890262 heterozygous sites being used for phasing (0 filtered, 0 indels excluded, 1890262 unphased)

#2. Retrieving reads that overlap heterozygous sites...
     file: samples/NA24143/rna/NA24143.name-clean.bam
          minimum mapq: 255
          mapping reads to variants...
               completed chromosome 1...
               completed chromosome 2...
               completed chromosome 3...
               completed chromosome 4...
               completed chromosome 5...
               completed chromosome 6...
               completed chromosome 7...
               completed chromosome 8...
               completed chromosome 9...
               completed chromosome 10...
               completed chromosome 11...
               completed chromosome 12...
               completed chromosome 13...
               completed chromosome 14...
               completed chromosome 15...
               completed chromosome 16...
               completed chromosome 17...
               completed chromosome 18...
               completed chromosome 19...
               completed chromosome 20...
               completed chromosome 21...
               completed chromosome 22...
          processing mapped reads...
          using alignment score cutoff of 126
          retrieved 1187348 reads
#3. Identifying connected variants...
     calculating sequencing noise level...
     sequencing noise level estimated at 0.000529
     creating read sets...
     generating read connectivity map...
     testing variant connections versus noise...
     285 variant connections dropped because of conflicting configurations (threshold = 0.010000)
     28443 variants covered by at least 1 read
#4. Identifying haplotype blocks...
#5. Phasing blocks...
#6. Outputting haplotypes...
#7. Outputting phased VCF...
     GT field is not being updated with phASER genome wide phase. This can be changed using the --gw_phase_vcf argument.
     Compressing and tabix indexing output VCF...

     COMPLETED using 1187348 reads in 839 seconds using 1 threads
     PHASED  7194 of 1890262 all variants (= 0.003806) with at least one other variant
     GENOME WIDE PHASED  0 of 1890262 unphased variants (= 0.000000)
     GENOME WIDE PHASE CORRECTED  0 of 1890262 variants (= 0.000000)
     Global maximum memory usage: 1129.78 (mb)

COMPLETED "Read backed phasing" of sample HG004 in 00:13:59 hh:mm:ss
DATE, TIME : 2019-01-07, 20:31:01

The End.
Time: 842.48 s
Mem:  1156896 Kb

NA24631 (HG005)
=====================================================

##################################################
              Welcome to phASER v1.1.0
  Author: Stephane Castel (scastel@nygenome.org)
  Updated by: Bishwa K. Giri (bkgiri@uncg.edu)
##################################################

Completed the check of dependencies and input files availability...

STARTED "Read backed phasing and ASE/haplotype analyses" ...
    DATE, TIME : 2019-01-07, 20:31:04
#1. Loading heterozygous variants into intervals...
Processing sample named HG005
    using all the chromosomes ...
    processing VCF...

    Memory efficient mode is deactivated...
    If RAM is limited, activate memory efficient mode using the flag "--process_slow = 1"...

     creating variant mapping table...
          1809385 heterozygous sites being used for phasing (0 filtered, 0 indels excluded, 1809385 unphased)

#2. Retrieving reads that overlap heterozygous sites...
     file: samples/NA24631/rna/NA24631.name-clean.bam
          minimum mapq: 255
          mapping reads to variants...
               completed chromosome 1...
               completed chromosome 2...
               completed chromosome 3...
               completed chromosome 4...
               completed chromosome 5...
               completed chromosome 6...
               completed chromosome 7...
               completed chromosome 8...
               completed chromosome 9...
               completed chromosome 10...
               completed chromosome 11...
               completed chromosome 12...
               completed chromosome 13...
               completed chromosome 14...
               completed chromosome 15...
               completed chromosome 16...
               completed chromosome 17...
               completed chromosome 18...
               completed chromosome 19...
               completed chromosome 20...
               completed chromosome 21...
               completed chromosome 22...
          processing mapped reads...
          using alignment score cutoff of 123
          retrieved 1720342 reads
#3. Identifying connected variants...
     calculating sequencing noise level...
     sequencing noise level estimated at 0.000571
     creating read sets...
     generating read connectivity map...
     testing variant connections versus noise...
     43 variant connections dropped because of conflicting configurations (threshold = 0.010000)
     51763 variants covered by at least 1 read
#4. Identifying haplotype blocks...
#5. Phasing blocks...
#6. Outputting haplotypes...
#7. Outputting phased VCF...
     GT field is not being updated with phASER genome wide phase. This can be changed using the --gw_phase_vcf argument.
     Compressing and tabix indexing output VCF...

     COMPLETED using 1720342 reads in 980 seconds using 1 threads
     PHASED  13617 of 1809385 all variants (= 0.007526) with at least one other variant
     GENOME WIDE PHASED  0 of 1809385 unphased variants (= 0.000000)
     GENOME WIDE PHASE CORRECTED  0 of 1809385 variants (= 0.000000)
     Global maximum memory usage: 1556.89 (mb)

COMPLETED "Read backed phasing" of sample HG005 in 00:16:20 hh:mm:ss
DATE, TIME : 2019-01-07, 20:47:24

The End.
Time: 983.30 s
Mem:  1594252 Kb
```

# TODO <_below_> :
### Testing the results with `error_rates.py`

In [ ]:
```bash
python ./githubX/error_rates.py /scratch1/haptreex_rnaseq/STAR-2.6.1d/giab_vcf/GM12878.vcf ./Hap_Runs/haptreex_GM12878/HapTreeX_withDASE_output.txt```

For GM12878, giab reference:

```
tool:            None
dataset:         HapTreeX_withDASE_output.txt
switch rate:     0
mismatch rate:   0
flat rate:       0
missing rate:    0
switch errors:   12
poss. switch:    549
mismatch errors: 128
poss. mismatch:  9857
flat errors:     149
poss. flat:      9857
phased count:    9896
num covered:     0
AN50:            0
N50:             0
max blk snp %:   0
runtime:         -1
missed vcf:      0
```

For GM24143,  giab ref:

```
tool:            None
dataset:         HapTreeX_withDASE_output.txt
switch rate:     0
mismatch rate:   0
flat rate:       0
missing rate:    0
switch errors:   4
poss. switch:    62
mismatch errors: 36
poss. mismatch:  1926
flat errors:     36
poss. flat:      1926
phased count:    7922
num covered:     0
AN50:            0
N50:             0
max blk snp %:   0
runtime:         -1
missed vcf:      0
```



For GM24149, giab reference:

tool:            None
dataset:         HapTreeX_withDASE_output.txt
switch rate:     0
mismatch rate:   0
flat rate:       0
missing rate:    0
switch errors:   2
poss. switch:    60
mismatch errors: 53
poss. mismatch:  1979
flat errors:     48
poss. flat:      1979
phased count:    7003
num covered:     0
AN50:            0
N50:             0
max blk snp %:   0
runtime:         -1
missed vcf:      0


For GM24386, giab reference:

tool:            None
dataset:         HapTreeX_withDASE_output.txt
switch rate:     0
mismatch rate:   0
flat rate:       0
missing rate:    0
switch errors:   7
poss. switch:    648
mismatch errors: 198
poss. mismatch:  11169
flat errors:     205
poss. flat:      11169
phased count:    13271
num covered:     0
AN50:            0
N50:             0
max blk snp %:   0
runtime:         -1
missed vcf:      0


For GM24631, giab reference:

tool:            None
dataset:         HapTreeX_withDASE_output.txt
switch rate:     0
mismatch rate:   0
flat rate:       0
missing rate:    0
switch errors:   3
poss. switch:    96
mismatch errors: 44
poss. mismatch:  3329
flat errors:     42
poss. flat:      3329
phased count:    14127
num covered:     0
AN50:            0
N50:             0
max blk snp %:   0
runtime:         -1
missed vcf:      0
